In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth', 1000) # to read records completely
import pickle
import numpy as np
import itertools
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle

## Data Prep

In [ ]:
# Importing the data scraped and pre-processed in part A
df = pd.read_pickle('data_clean.pkl')

In [ ]:
models = pd.read_csv('models.csv', header=None, names = ['brand', 'model'])
models['brand'] = models['brand'].str.replace(r'[^\w\s]+', '', regex=True) # clean some punctuation errors (nissan.)
models = models[~models.brand.isin(['car', 'sedan', 'problem'])] # remove rows that are not brands

In [ ]:
df.Message.str.contains('said').sum()

In [ ]:
#.lower() on all words
df.Message_words = df.Message_words.apply(lambda x: [i.lower() for i in x])

## Replace models with brands

In [ ]:
messages = df.Message_words.to_list()

In [ ]:
# numpy.where to do the replacement. Replaces only first appearance of brand.
# ~ 5-10 minutes of execute
messages2 = []
t = 0
for m in messages:
    for i,row in models.iterrows():
        m = np.where(m == row['model'], row['brand'], m)
        
    t+=1
    
    #print(t)
        
    messages2.append(m)
        
df['Message_words_v2'] = messages2

In [ ]:
# let's find the occurences of changes
m1 = df.Message_words.to_list()
m2 = df.Message_words_v2.to_list()

c = 0
W = 0
for i in range(len(m2)):
    for j in range(len(m2[i])):
        if m1[i][j] != m2[i][j]: #word by word comparison
            c +=1
        W += 1 # if we want to count all words

In [ ]:
print(f'{c} changes of models')

In [ ]:
df[['Message_words', 'Message_words_v2']].head(10)

In [ ]:
#Now we have cases where the text says "nissan", "nissan" because we replaced the model. Let's clean that
#Removing duplicates in lists while keeping order
def remove_consecutive_duplicate (text): #removes consecutive duplicates
    return np.array([i for i, j in itertools.groupby(text)])

df["Message_words_v3"] = df["Message_words_v2"].apply(remove_consecutive_duplicate)

## Link attributes to models

In [ ]:
# first lets count brand mentions in each record
#Get a unique list of brands
brands_list = models.brand.drop_duplicates().to_list()

brands_list.extend(['lexus', 'ferrari', 'merzedesbenz', 'tesla','gm', 'peugeot', 'jeep', 'bentley', 'fiat'])

In [ ]:
messages = df.Message_words_v3.to_list()

#Get a nested list of brands mentioned in reviews
brands_in_message = []
for m in messages:
    
    brands_mentioned = []
    for brand in brands_list:
        if len(np.where(m == brand)[0]) >0:
            brands_mentioned.append(brand)
            
    brands_in_message.append(brands_mentioned)
        
    

In [ ]:
df['Brands_in_message'] = brands_in_message
df['Qty_brands'] = df.Brands_in_message.apply(lambda x: len(x))
df['Qty_brands'].value_counts()

# 1291 messages don't mention any brand (remove them from analysis?)
# Most messages discuss a single brand
# Also common to compare 2 brands

In [ ]:
df.loc[df.Qty_brands == 0, 'Message_words_v3'].head(10) # to review records with no brand mentions
# most of them are for seeking advice

We can assign attributes mentioned in the reviews, like the power of a car, to a specific brand. For example, in the sentence "I like the BMW for its power. On the other hand, the Honda is reliable" we would want to assign the the attribute "power" to "BMW" and "reliable" to "Honda". We have created functions for two different approaches:
1) assign all words found between 1st brand mention and next brand mention to 1st brand

2) assign n words to each side of the brand mention to the brand. We found n = 4 to work best. 

In [ ]:
# First approach: link all words found between brand mention and next brand mention.
# Except for first brand which also receives words since beginning of message

m = df.Message_words_v3[9] #record 9 is a good exaple

def get_attr_in_the_right(m, brand_list):

    brand_dic = {}
    for brand in brands_list:
        ix = np.where(m == brand)[0]
        if len(ix) > 0:
            brand_dic[brand] = ix[0]


    brand_dic = dict(sorted(brand_dic.items(), key=lambda x:x[1]))

    brand_list = list(brand_dic.values()) + [len(m)]

    review={}
    for i, tup in enumerate(brand_dic):
#         print(brand_dic[tup])
#         print(tup)
        
        if i == 0:
            
            review[tup] = m[0:brand_list[1]]
        else:
            review[tup] = m[brand_list[i]:brand_list[i+1]]
            
    return review



get_attr_in_the_right(m, brands_list) #was brand_list, ASK CARLOS    

In [ ]:
# now we run it for all records

brand_reviews_1st_approach = []

for m in df.Message_words_v3:

    brand_reviews_1st_approach.append(get_attr_in_the_right(m, brands_list))

# generates a list (1 entry per row) of dictionaries {brand_1: part of text corresponding, 
# brand_2: part of text corresponding,}

In [ ]:


review_accum_1st = {i:[] for i in brands_list} #deprecated dictionary but still interesting

for brand in brands_list:
    for review in brand_reviews_1st_approach:
        for single_review in review:
            #print(review[single_review])
            if single_review==brand:
                review_accum_1st[brand].extend(review[single_review])

## Task E: Finding Aspirational Brands
Approach: Measure "ASPIRATION" as a distance between **their market share rank** and their **positive-comments share rank**

Step 1: Get a list of positive-sentiment words and count appearances for each brand

In [ ]:
#import positive attributes
pos_df = pd.read_csv('positive_indicator.csv') # words manually tagged as positive

pos_attributes = list(pos_df[pos_df.positive_indicator == 1].word)
brands = list(review_accum_1st.keys())

pos_attribute_count = []

for brand in brands: 
    count = 0
    for i in pos_attributes: 
        for j in review_accum_1st[brand]: 
            if j == i:
                count += 1
    
    pos_attribute_count.append(count)


results_df = pd.DataFrame({'brand': brands, 'count': pos_attribute_count})
results_df['rel_freq'] = results_df['count']*100/results_df['count'].sum()
results_df.sort_values('count', ascending= False).reset_index(drop=True)


# we remove brands with count = 0 and add a rank columns
results_df = results_df[results_df['count'] > 0 ]

results_df['pos_comment_rank'] = results_df['rel_freq'].rank(ascending=False)

Step 2: Get brand share info from an external datasource and rank the brands according to their market share

 - Source: GoodCarBadCar, 2019:  https://www.goodcarbadcar.net/2019-u-s-auto-sales-figures-by-brand/

In [ ]:
df_share = pd.read_excel('brand_share.xlsx')
df_share.Brand = df_share.Brand.str.lower()
df_share['rel_share'] = df_share.YTD*100/df_share.YTD.sum()
df_share['share_rank'] = df_share.rel_share.rank(ascending = False)

Step 3: Merge both data sources and look for largest distances (deltas)

In [ ]:
df_share = df_share.merge(results_df, left_on = 'Brand', right_on = 'brand')
df_share

In [ ]:
df_share['dif_in_rank'] = df_share.share_rank - df_share.pos_comment_rank

In [ ]:
df_share['dif_in_rank']

In [ ]:
df_share.sort_values('dif_in_rank', ascending=False)[[
    'Brand','share_rank', 'pos_comment_rank','dif_in_rank']].reset_index(drop=True)